In [2]:
###############
## FUNCTIONS ##
###############

def normalise(img):
    img_new = np.copy(img).astype(float)  # Ensure image is float for normalization
    num_channels = img_new.shape[2]
    for c in range(num_channels):
        channel = img_new[:, :, c]
        min_val = channel.min()
        max_val = channel.max()
        if max_val > min_val:  # Avoid division by zero
            img_new[:, :, c] = (channel - min_val) / (max_val - min_val)
        else:
            img_new[:, :, c] = 0  # Set to 0 if channel is flat
    return (img_new * 255).astype(np.uint8)  # Scale back to [0, 255]

def apply_clahe(img, clip_limit=2.0, tile_grid_size=(8, 8)):
    """
    Apply CLAHE to enhance contrast in the image.
    
    Parameters:
    - img: Input image (RGB or grayscale).
    - clip_limit: Threshold for contrast limiting.
    - tile_grid_size: Size of the grid for CLAHE.
    
    Returns:
    - Image with CLAHE applied.
    """
    ## Convert to grayscale if the image is RGB
    if len(img.shape) == 3 and img.shape[2] == 3:  # Check if RGB
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray_img = img  # Already grayscale
    
    ## Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    
    ## Apply CLAHE
    clahe_img = clahe.apply(gray_img)
    
    return clahe_img


def load_images_from_path(path):
    """
    Goal: Load images from path.
    Args: 
    - path (str): Path to folder containing images.
    Outputs:
    - imgs (list(np.array)): The images.
    - fnames (list(str)): The names of the images.
    """
    imgs = []
    fnames = []
    for f in os.listdir(path):        
        if f[0] != ".":
            # print("Loading " + f)
            img_bgr = cv2.imread(path + f)
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            imgs += [img_rgb]
            fnames += [f]
    return imgs, fnames

def load_masks_from_path(path):
    """
    Goal: Load masks from path.
    Args: 
    - path (str): Path to folder containing images.
    Outputs:
    - imgs (list(np.array)): The images.
    - fnames (list(str)): The names of the images.
    """
    masks = []
    fnames = []
    for f in os.listdir(path):        
        if f[0] != ".":
            # print("Loading " + f)
            mask = cv2.imread(path + f)            
            masks += [mask]
            fnames += [f]
    return masks, fnames

def show_image(img, fig_size):
    """
    Goal: Wrapper function to quickly visualise images.
    Args:
    - img (np.array): image.
    """ 
    fig = plt.figure(figsize=(fig_size,fig_size))
    plt.imshow(img)
    plt.tight_layout()
    plt.show()
    plt.close()

def orient_object_pca(image, mask):
    """
    Goal: Orient object horizontally.
    Inputs:
    image (np.array): Image to orient.
    mask (np.array): Binary mask of the target object of the same dimension as the image.
    Outputs:
    rotate (np.array): The rotate image.
    """    
    ## Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    ## Filter out small contours based on area
    contours = [contour for contour in contours if cv2.contourArea(contour) > 500]
    
    if not contours:
        print("No significant contours found!")
        return image
    
    ## Assume the largest contour is the fish
    largest_contour = max(contours, key=cv2.contourArea)
    
    ## Compute the centroid of the fish
    moments = cv2.moments(largest_contour)
    if moments['m00'] == 0:  # To avoid division by zero
        return image
    
    ## Calculate the centroid (cx, cy)
    cx = int(moments['m10'] / moments['m00'])
    cy = int(moments['m01'] / moments['m00'])
    
    ## Perform PCA on the contour points
    contour_points = largest_contour.reshape(-1, 2).astype(np.float32)
    mean, eigenvectors = cv2.PCACompute(contour_points, mean=np.array([]))
    
    ## The angle of the first principal component
    angle = np.arctan2(eigenvectors[0, 1], eigenvectors[0, 0]) * 180 / np.pi
    
    ## Rotate the image to align the fish horizontally
    (h, w) = image.shape[:2]
    center = (cx, cy)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    
    return rotated


#
###

In [3]:
##############
## INITIATE ##
##############

## Imports
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import torch

## Paths
PT_ROOT = '/Users/willembonnaffe/Library/CloudStorage/GoogleDrive-willem.bonnaffe@gmail.com/My Drive/Colab Notebooks/IECDT/guppy_project_V1/'
PTI_IMAGES = PT_ROOT + 'data/images/'
PTI_MASKS_FIN = PT_ROOT + 'data/masks_caudal_fin/'
PTI_MASKS_BODY = PT_ROOT + 'data/masks_main_body/'
PTI_MASKS_EYE = PT_ROOT + 'data/masks_eye/'
PTI_MASKS_RULER = PT_ROOT + 'data/masks_ruler/'
PTO = PT_ROOT + 'outputs/handcrafted_features/'

## Check if path to output exists
if not os.path.exists(PTO):
    os.mkdir(PTO)

#
###

/Users/willembonnaffe/anaconda3/envs/pytorch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
#################
## LOAD IMAGES ##
#################

## Path 
imgs, fnames = load_images_from_path(PTI_IMAGES)

## Check that images are horizontal
imgs_new = []
for img in imgs:
    if img.shape[0] > img.shape[1]:
        img_new = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    else:
        img_new = img
    imgs_new += [img_new]
imgs = imgs_new
        
#
###

################
## LOAD MASKS ##
################

## Get masks
masks_fin, fnames_masks = load_masks_from_path(PTI_MASKS_FIN)
masks_body, _ = load_masks_from_path(PTI_MASKS_BODY)
masks_eye, _ = load_masks_from_path(PTI_MASKS_EYE)
masks_ruler, _ = load_masks_from_path(PTI_MASKS_RULER)

#
###

######################################
## CHECK THAT IMAGES ARE HORIZONTAL ##
######################################

## Check that images are horizontal
imgs_new = []
masks_fin_new = []
masks_body_new = []
masks_eye_new = []
masks_ruler_new = []

for k, img in enumerate(imgs):
    if img.shape[0] > img.shape[1]:
        ## Rotate image
        img_new = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        
        ## Rotate corresponding masks
        mask_fin_new = cv2.rotate(masks_fin[k], cv2.ROTATE_90_CLOCKWISE)
        mask_body_new = cv2.rotate(masks_body[k], cv2.ROTATE_90_CLOCKWISE)
        mask_eye_new = cv2.rotate(masks_eye[k], cv2.ROTATE_90_CLOCKWISE)
        mask_ruler_new = cv2.rotate(masks_ruler[k], cv2.ROTATE_90_CLOCKWISE)
    else:
        ## Keep image and masks as is
        img_new = img
        mask_fin_new = masks_fin[k]
        mask_body_new = masks_body[k]
        mask_eye_new = masks_eye[k]
        mask_ruler_new = masks_ruler[k]
    
    ## Append updated images and masks
    imgs_new.append(img_new)
    masks_fin_new.append(mask_fin_new)
    masks_body_new.append(mask_body_new)
    masks_eye_new.append(mask_eye_new)
    masks_ruler_new.append(mask_ruler_new)

## Replace original lists with updated ones
imgs = imgs_new
masks_fin = masks_fin_new
masks_body = masks_body_new
masks_eye = masks_eye_new
masks_ruler = masks_ruler_new

#
###

In [5]:
## FEATURE TABLE
FEATURES = [['FILE_NAME', 'LENGTH_CENTRE', 'WIDTH_CENTRE', 'LENGTH_MINMAX', 'WIDTH_MINMAX', 'FIN_AREA', 'BODY_AREA', 'EYE_AREA']]

## For each image and masks
for img_index in range(len(imgs)):
    
    ####################
    ## APPLY TO IMAGE ##
    ####################
    
    ## Select image and masks
    img = imgs[img_index]
    mask_fin = (masks_fin[img_index] > 100)
    mask_body = (masks_body[img_index] > 100)
    mask_eye = (masks_eye[img_index] > 100)
    mask_ruler = (masks_ruler[img_index] > 100)
    
    #
    ###
    
    ###################
    ## COMBINE MASKS ##
    ###################
    
    ## Compute combined masks
    h, w, c = img.shape
    masks = np.zeros((h, w, c))
    masks[:,:,0] = mask_fin[:,:,0]
    masks[:,:,1] = (mask_body[:,:,0])*1 - (mask_eye[:,:,0])*1 
    masks[:,:,2] = mask_eye[:,:,0]
    masks = masks.astype(np.uint8)
    
    #
    ###
    
    ##################
    ## ORIENT RULER ##
    ##################
    
    ## Image and mask
    image = (img * mask_ruler).astype(np.uint8)
    mask = (mask_ruler[:,:,0] * 255).astype(np.uint8)
    
    ## Orient the fish using PCA
    oriented_image_pca = orient_object_pca(image, mask)
    oriented_mask_pca = orient_object_pca(mask, mask)
    
    #
    ###
    
   ###################
    ## COMPUTE MMPPX ##
    ###################

    """ Old method
    ## Binarise ruler
    img_bin = ((normalise(oriented_image_pca)[:,:,0]>100)*1).astype(np.uint8)
    
    ## Compute strip of ruler
    idx_h = int(np.mean(np.where((img_bin.sum(1)>1) > 0.5))) ## Center of ruler along height axis
    idx_w = int(np.mean(np.where((img_bin.sum(0)>1) > 0.5))) ## Center of ruler along width axis
    img_bin_strip = img_bin[idx_h,idx_w-1000:idx_w+1000]
    
    ## Compute number of ticks
    length_in_mm = (np.diff(img_bin_strip)==1).sum()
    length_in_px = img_bin_strip.shape[0]
    mmppx = length_in_mm/length_in_px
    """

    ## Binarise ruler
    img_bin = apply_clahe(normalise(oriented_image_pca))
    # img_bin = ((apply_clahe(normalise(oriented_image_pca))>50)*1).astype(np.uint8)
    
    ## Compute strip of ruler
    idx_h = int(np.mean(np.where((img_bin.sum(1)>1) > 0.5))) ## Center of ruler along height axis
    idx_w = int(np.mean(np.where((img_bin.sum(0)>1) > 0.5))) ## Center of ruler along width axis
    # img_bin_strip = img_bin[idx_h,idx_w-1000:idx_w+1000]
    img_bin_marg = img_bin.sum(0) 
    img_bin_strip = (img_bin_marg < img_bin_marg.mean())[idx_w-1000:idx_w+1000]
    
    ## Compute number of ticks
    length_in_mm = (np.diff(img_bin_strip.astype(np.int16))==1).sum()
    length_in_px = img_bin_strip.shape[0]
    mmppx = length_in_mm/length_in_px
    
    #
    ###
    
    ##################
    ## ORIENT GUPPY ##
    ##################
    
    ## Image and mask
    image = (img * mask_body).astype(np.uint8)
    mask = (mask_body[:,:,0] * 255).astype(np.uint8)
    
    ## Orient the fish using PCA
    oriented_image_pca = orient_object_pca(image, mask)
    oriented_mask_pca = orient_object_pca(mask, mask)
    
    #
    ###
    
    #############################
    ## COMPUTE STANDARD LENGTH ##
    #############################
    
    ## Binarise image
    img_bin = ((oriented_mask_pca)>100)*1
    
    ## Compute strip of guppy
    idx_h = int(np.mean(np.where((img_bin.sum(1)>1) > 0.5))) ## Center of guppy along height axis
    # idx_w = int(np.mean(np.where((img_bin.sum(0)>1) > 0.5))) ## Center of guppy along width axis
    img_bin_strip = img_bin[idx_h]
    
    ## Compute left and right bound
    pts_in_fish = np.argwhere(img_bin_strip == 1)
    LB = np.min(pts_in_fish)
    RB = np.max(pts_in_fish)
    
    ## Compute standard length
    length_in_px = RB - LB
    length_in_mm = length_in_px * mmppx

    ## Store
    LENGTH_CENTRE = length_in_mm
    
    #
    ###
    
    ############################
    ## COMPUTE STANDARD WIDTH ##
    ############################
    
    ## Binarise image
    img_bin = ((oriented_mask_pca)>100)*1
    
    ## Compute strip of guppy
    # idx_h = int(np.mean(np.where((img_bin.sum(1)>1) > 0.5))) ## Center of guppy along height axis
    idx_w = int(np.mean(np.where((img_bin.sum(0)>1) > 0.5))) ## Center of guppy along width axis
    # img_bin_strip = img_bin[idx_h]
    img_bin_strip = img_bin[:,idx_w]
    
    ## Compute left and right bound
    pts_in_fish = np.argwhere(img_bin_strip == 1)
    LB = np.min(pts_in_fish)
    RB = np.max(pts_in_fish)
    
    ## Compute standard length
    width_in_px = RB - LB
    width_in_mm = width_in_px * mmppx

    ## Store
    WIDTH_CENTRE = width_in_mm
    
    #
    ###
    
    ###########################
    ## COMPUTE MINMAX LENGTH ##
    ###########################
    
    ## Binarise image
    img_bin = ((oriented_mask_pca)>100)*1
    
    ## Compute strip of guppy
    img_bin_strip = (img_bin.sum(0)>1)*1
    
    ## Compute left and right bound
    pts_in_fish = np.argwhere(img_bin_strip == 1)
    LB = np.min(pts_in_fish)
    RB = np.max(pts_in_fish)
    
    ## Compute standard length
    length_in_px = RB - LB
    length_in_mm = length_in_px * mmppx

    ## Store
    LENGTH_MINMAX = length_in_mm
    
    #
    ###
    
    ##########################
    ## COMPUTE MINMAX WIDTH ##
    ##########################
    
    ## Binarise image
    img_bin = ((oriented_mask_pca)>100)*1
    
    ## Compute strip of guppy
    img_bin_strip = (img_bin.sum(1)>1)*1
    
    ## Compute left and right bound
    pts_in_fish = np.argwhere(img_bin_strip == 1)
    LB = np.min(pts_in_fish)
    RB = np.max(pts_in_fish)
    
    ## Compute standard length
    width_in_px = RB - LB
    width_in_mm = width_in_px * mmppx
    
    ## Store
    WIDTH_MINMAX = width_in_mm
    
    #
    ###
    
    ##################
    ## COMPUTE AREA ##
    ##################
    
    ## Areas
    fin_area, body_area, eye_area = masks.sum((0,1)) * mmppx ** 2

    ## STORE
    FIN_AREA = fin_area
    BODY_AREA = body_area
    EYE_AREA = eye_area
    
    #
    ###

    #############
    ## COLLECT ##
    #############

    FEATURES_ = [fnames[img_index], LENGTH_CENTRE, WIDTH_CENTRE, LENGTH_MINMAX, WIDTH_MINMAX, FIN_AREA, BODY_AREA, EYE_AREA]
    FEATURES += [FEATURES_]

    #
    ###

In [6]:
##################
## END AND SAVE ##
##################

## Save table
np.savetxt(PTO + 'phenotypes.csv', FEATURES, fmt='%s', delimiter=',')

#
###